In [ ]:
import pandas as pd 
import numpy as np 
import os 
import pathlib 
from tqdm import tqdm 

import re

from helper_functions import * 
path_to_all_fa = "/Volumes/HNSD01/storage/ref/hg19"
outdir = "/Volumes/HNSD01/outdir"
# outdir = "/Volumes/HNSD03/outdir"

outputdir = os.path.join(outdir, "output_methyl_ReadBased_Models")
path_to_01_output = os.path.join(outputdir, "01_output")
os.system(f"mkdir -p {path_to_01_output}")

inputdir = "/Volumes/HNWD02/outdir/TSMA_validation/reads"

# panel = "TSMA_panel"
# region = "1_121261113_121261188"
full_paneldf = pd.read_excel("./assets/TSMA panel.xlsx", skiprows=2)
full_paneldf.columns = ["chrom", "start", "end", "annotation", "panel"]
full_paneldf["panel"] = full_paneldf["panel"].apply(lambda x: x.replace(" ", "_"))
full_paneldf["regionname"] = full_paneldf[["chrom", "start", "end"]].apply(lambda x: f"{x[0]}_{x[1]}_{x[2]}", axis = 1)

 # ***** helper function: preprocess input file *****
def preprocess(input_file, region):
    region_chrom = f"chr{region.split('_')[0]}"
    region_start = int(region.split("_")[1])
    region_end = int(region.split("_")[2])

    refseq_at_cluster = get_refseq(path_to_all_fa = path_to_all_fa, 
                                    chrom = region_chrom, 
                                    start = region_start, 
                                    end = region_end + 1)
    all_cpg_in_cluster = [m.start(0) for m in re.finditer("CG", refseq_at_cluster)]
    cpg_coords = [item + region_start for item in all_cpg_in_cluster]

    df = pd.read_csv(input_file, index_col = [0])
    if df.shape[0] != 0:
        pattern = re.compile("^[1-9][0-9]M")
        df["check_cigar"] = df["cigar"].apply(lambda x: bool(pattern.fullmatch(x)))
        df = df[df["check_cigar"] == True]
        if df.shape[0] != 0:            
            df["end"] = df[["start", "cigar"]].apply(lambda x: int(x[0]) + int(x[1].replace("M", "")), axis = 1)
            
            for cpg_pos in cpg_coords:
                df[cpg_pos] = df[["start", "end", "seq"]].apply(lambda x: get_CpG_status(x[0], x[1], x[2], cpg_pos, mode = "num"), axis = 1)
                
            betadf = pd.DataFrame(data = df["sample"].unique())
            betadf.columns = ["sample"]
            for cpg_pos in cpg_coords:    
                tmpdf = df[["sample", cpg_pos]].copy()
                tmpcountdf = tmpdf.groupby('sample')[cpg_pos].apply(lambda x: (x == 1).sum()/((x == 0).sum() + (x == 1).sum()) ).reset_index(name= "meth_level_{}".format(cpg_pos))
                betadf = betadf.merge(tmpcountdf[["sample", "meth_level_{}".format(cpg_pos)]], right_on = "sample", left_on = "sample", how = "outer")
        else:
            df = pd.DataFrame()
            betadf = pd.DataFrame()
            cpg_coords = []
    else:
        df = pd.DataFrame()
        betadf = pd.DataFrame()
        cpg_coords = []
    return df, betadf, cpg_coords

# all_panels = full_paneldf.panel.unique()
all_panels = ["TSMA_panel"]
for panel in all_panels:
    for region in full_paneldf[full_paneldf["panel"] == panel].regionname.unique():
        path_to_save_output = os.path.join(path_to_01_output, panel, region)
        # path_to_check_01_output = "/Volumes/HNWD02/outdir/output_methyl_ReadBased_Models/01_output"
        # path_to_check_01_output = "/Volumes/HNSD01/outdir/output_methyl_ReadBased_Models/01_output"
        path_to_check_01_output = "/Volumes/HNWD02/outdir/output_methyl_ReadBased_Models/01_output"

        path_to_check_output = os.path.join(path_to_check_01_output, panel, region)

        # if os.path.isfile(os.path.join(path_to_save_output, "betadf.csv")) == False:
        if os.path.isfile(os.path.join(path_to_check_output, "betadf.csv")) == False:
            os.system(f"mkdir -p {path_to_save_output}")
            all_files = [item for item in pathlib.Path(inputdir).glob(f"*/{panel}/*{region}.csv")]

            metadata = pd.read_excel("./metadata/metadata_TSMA_R8281_R8282.xlsx")[["LABCODE", "TYPE"]]
            tsmadf = pd.read_excel("./assets/12967_2024_5416_MOESM1_ESM.xlsx")

            betadf = pd.DataFrame()
            for input_file in tqdm(all_files):
                sampleid = str(input_file).split("/")[-3].split("_")[0].split("-")[1]
                tmpdf, tmp_betadf, cpg_coords = preprocess(input_file, region)
                if tmpdf.shape[0] != 0:
                    betadf = pd.concat([betadf, tmp_betadf], axis = 0)
                    tmpdf.to_csv(os.path.join(path_to_save_output, f"{sampleid}.tmpdf.csv"))
                    tmpdf_filtered = tmpdf[tmpdf[cpg_coords].isin([0, 1]).all(axis=1)]
            if betadf.shape[0] != 0:
                betadf["LABCODE"] = betadf["sample"].apply(lambda x: x.split("_")[0].split("-")[1])
                betadf = betadf.merge(metadata, right_on = "LABCODE", left_on = "LABCODE")
            betadf.to_csv(os.path.join(path_to_save_output, "betadf.csv"))
        else:
            print(f"File {os.path.join(path_to_save_output, 'betadf.csv')} exists")
